In [108]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from option_pricing import AmericanOption

In [109]:
date = pd.to_datetime('2013-06-03')

In [110]:
option_data = pd.read_csv('./data/option_data/2013-06-03options.csv')
stock_data = pd.read_csv('./data/stock_data/2013-06-03stocks.csv')

In [111]:
option_data['expiration'] = pd.to_datetime(option_data['expiration'])

In [112]:
option_data['time_to_expiry'] = (option_data['expiration'] - date).dt.days/252
option_data['mid_price'] = (option_data['ask']+option_data['bid'])/2

In [113]:
total_data = option_data.merge(stock_data[['symbol', 'close']],
                         left_on='underlying',
                         right_on='symbol')

In [114]:
total_data = total_data.loc[total_data['style']=='A']

In [115]:
total_data = total_data[['underlying', 'expiration', 'type', 'strike', 'mid_price', 'close', 'implied_volatility', 'time_to_expiry']]

In [116]:
total_data = total_data.iloc[:1000]

In [117]:
total_data['binomial_price'] = total_data.apply(lambda row: AmericanOption(S0=row.close, 
                                                                           K=row.strike,
                                                                           r=0.05,
                                                                           T=row.time_to_expiry,
                                                                           sigma=row.implied_volatility, 
                                                                           option_type=row.type
                                                                          ).price_binomial(), axis=1)

In [118]:
total_data['abs_err'] = np.abs(total_data['binomial_price'] - total_data['mid_price'])
total_data['signed_err'] = total_data['binomial_price'] - total_data['mid_price']
total_data['pct_err'] = total_data['abs_err'] / (total_data['mid_price'].abs() + 1e-8)

In [119]:
mae  = total_data['abs_err'].mean()
rmse = np.sqrt((total_data['signed_err']**2).mean())
mape = 100 * total_data['pct_err'].mean()
bias = total_data['signed_err'].mean()

In [120]:
print(f"MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%, Bias={bias:.4f}")

MAE=0.6423, RMSE=1.1894, MAPE=325400074.24%, Bias=0.5176


In [121]:
total_data['moneyness'] = total_data['close'] / total_data['strike']

total_data['moneyness_bin'] = pd.cut(total_data['moneyness'], bins=[0,0.9,0.97,1.03,1.1, np.inf],
                             labels=['deep OTM','OTM','ATM','ITM','deep ITM'])

total_data['mat_days'] = (total_data['time_to_expiry'] * 252).clip(lower=0).astype(int)

total_data['mat_bin'] = pd.cut(total_data['mat_days'], bins=[-1,7,30,90,365,9999],
                       labels=['<1w','1w-1m','1m-3m','3m-1y','>1y'])

In [124]:
summary = total_data.groupby(['moneyness_bin','mat_bin']).agg(
    count=('abs_err','size'),
    mae=('abs_err','mean'),
    rmse=('signed_err', lambda x: np.sqrt((x**2).mean())),
    median_err=('signed_err','median')
).reset_index()

summary

,moneyness_bin,mat_bin,count,mae,rmse,median_err
0,deep OTM,<1w,28,0.628100,1.438605,-0.00415
1,deep OTM,1w-1m,36,0.104833,0.164708,-0.01945
2,deep OTM,1m-3m,58,0.152095,0.230777,-0.00760
3,deep OTM,3m-1y,96,0.373146,0.547321,0.01120
4,deep OTM,>1y,30,1.920603,2.376080,0.08860
5,OTM,<1w,18,0.386467,0.497973,0.23470
6,OTM,1w-1m,16,0.177175,0.232066,0.13605
7,OTM,1m-3m,30,0.235413,0.333219,0.10410
8,OTM,3m-1y,34,0.709041,0.998676,0.10870
9,OTM,>1y,6,2.822517,3.326225,1.34865
